In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm --no-cache-dir

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 116.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=d23ecb10680e435917e6eb623f85ae4b999b98d650ae1c58f8569a43a3bf73a1
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled tor

In [ ]:
!git clone https://github.com/ygong0712/Att-UMamba

Cloning into 'Att-UMamba'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 403 (delta 148), reused 265 (delta 55), pack-reused 0
Receiving objects: 100% (403/403), 6.27 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
%cd /content/Att-UMamba/umamba

/content/Att-UMamba/umamba


In [ ]:
!pip install -e .

In [ ]:
import torch
import mamba_ssm
import os

In [ ]:
!mkdir -p /content/Att-UMamba/data/nnUNet_raw
!mkdir -p /content/Att-UMamba/data/nnUNet_preprocessed

In [ ]:
!cp -rp /content/drive/MyDrive/nnUNet/U-Mamba/nnUNet_raw/Dataset600_PSMA /content/Att-UMamba/data/nnUNet_raw/Dataset600_PSMA
!cp -rp /content/drive/MyDrive/nnUNet/U-Mamba/nnUNet_preprocessed/Dataset600_PSMA /content/Att-UMamba/data/nnUNet_preprocessed/Dataset600_PSMA

## Preprocess if needed

In [ ]:
!nnUNetv2_plan_and_preprocess -d 600 --verify_dataset_integrity

Fingerprint extraction...
Dataset501_Glioblastoma
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 117/117 [00:27<00:00,  4.23it/s]
Experiment planning...
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 105, 'patch_size': array([192, 160]), 'median_image_size_in_voxels': array([172., 137.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': (2, 2, 2, 2, 2, 2), 'n_conv_per_stage_decoder': (2, 2, 2, 2, 2), 'num

## Train

In [ ]:
!nnUNetv2_train 600 3d_fullres all -tr nnUNetTrainerUMambaBot

2024-08-11 17:07:54.132199: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 17:07:54.157903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 17:07:54.182191: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 17:07:54.189386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 17:07:54.210554: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
!nnUNetv2_train 600 3d_fullres all -tr nnUNetTrainerUMambaEnc


Dataset600 PSMA:

nnU-Net: best: 0.6485 (100, 50, 25)

U-Mamba Bot: best: 0.6503  (100, 50, 25)

Attention U-Mamba: best: 0.6552  (100, 50, 25)

Attention U-Mamba: best: 0.6746 (200, 50, 25)


Dataset501:


Dataset701: 0.8633

Dataset702: 0.8623

In [ ]:
!nnUNetv2_predict -i /content/Att-UMamba/data/nnUNet_raw/Dataset600_PSMA/imagesTs -o /content/Att-UMamba/data/nnUNet_results/Dataset600_PSMA/nnUNetTrainerUMambaBot__nnUNetPlans__3d_fullres/inference -d 600 -c 3d_fullres -f all -tr nnUNetTrainerUMambaBot --disable_tta


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-08-11 20:48:03.133904: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 20:48:03.703368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 20:48:03.968223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] U

In [ ]:
!nnUNetv2_predict -i /content/Att-UMamba/data/nnUNet_raw/Dataset600_PSMA/imagesTs -o /content/Att-UMamba/data/nnUNet_results/Dataset600_PSMA/nnUNetTrainerUMambaEnc__nnUNetPlans__3d_fullres/inference -d 600 -c 3d_fullres -f all -tr nnUNetTrainerUMambaEnc --disable_tta

In [ ]:
!cp -rp /content/Att-UMamba/data/nnUNet_results/Dataset600_PSMA /content/drive/MyDrive/nnUNet/U-Mamba/nnUNet_results_50_epoch

## Evaluation

###DSC

In [ ]:
import numpy as np
import nibabel as nib
import pathlib as plb
import cc3d
import csv
import sys


def nii2numpy(nii_path):
    # input: path of NIfTI segmentation file, output: corresponding numpy array and voxel_vol in ml
    mask_nii = nib.load(str(nii_path))
    mask = mask_nii.get_fdata()
    pixdim = mask_nii.header['pixdim']
    voxel_vol = pixdim[1]*pixdim[2]*pixdim[3]/1000
    return mask, voxel_vol


def con_comp(seg_array):
    # input: a binary segmentation array output: an array with seperated (indexed) connected components of the segmentation array
    connectivity = 18
    conn_comp = cc3d.connected_components(seg_array, connectivity=connectivity)
    return conn_comp


def false_pos_pix(gt_array,pred_array):
    # compute number of voxels of false positive connected components in prediction mask
    pred_conn_comp = con_comp(pred_array)

    false_pos = 0
    for idx in range(1,pred_conn_comp.max()+1):
        comp_mask = np.isin(pred_conn_comp, idx)
        if (comp_mask*gt_array).sum() == 0:
            false_pos = false_pos+comp_mask.sum()
    return false_pos



def false_neg_pix(gt_array,pred_array):
    # compute number of voxels of false negative connected components (of the ground truth mask) in the prediction mask
    gt_conn_comp = con_comp(gt_array)

    false_neg = 0
    for idx in range(1,gt_conn_comp.max()+1):
        comp_mask = np.isin(gt_conn_comp, idx)
        if (comp_mask*pred_array).sum() == 0:
            false_neg = false_neg+comp_mask.sum()

    return false_neg


def dice_score(mask1,mask2):
    # compute foreground Dice coefficient
    overlap = (mask1*mask2).sum()
    sum = mask1.sum()+mask2.sum()
    dice_score = 2*overlap/sum
    return dice_score



def compute_metrics(nii_gt_path, nii_pred_path):
    # main function
    gt_array, voxel_vol = nii2numpy(nii_gt_path)
    pred_array, voxel_vol = nii2numpy(nii_pred_path)

    false_neg_vol = false_neg_pix(gt_array, pred_array)*voxel_vol
    false_pos_vol = false_pos_pix(gt_array, pred_array)*voxel_vol
    dice_sc = dice_score(gt_array,pred_array)

    return dice_sc, false_pos_vol, false_neg_vol




if __name__ == "__main__":

    nii_gt_path = '/content/drive/MyDrive/nnUNet/U-Mamba/nnUNet_raw/Dataset600_PSMA/labelsTs'
    nii_pred_path = '/content/drive/MyDrive/nnUNet/U-Mamba/nnUNet_results/Dataset600_PSMA/nnUNetTrainerUMambaBot__nnUNetPlans__3d_fullres/inference'

    nii_gt_path = plb.Path(nii_gt_path)
    nii_pred_path = plb.Path(nii_pred_path)
    dice_sc, false_pos_vol, false_neg_vol = compute_metrics(nii_gt_path, nii_pred_path)

    csv_header = ['gt_name', 'dice_sc', 'false_pos_vol', 'false_neg_vol']
    csv_rows = [nii_gt_path.name,dice_sc, false_pos_vol, false_neg_vol]

    with open("metrics.csv", "w", newline='') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(csv_header)
        writer.writerows(csv_rows)

###NSD

In [ ]:
from scipy.spatial.distance import directed_hausdorff

def load_nifti(file_path):
    img = nib.load(file_path)
    data = img.get_fdata()
    return data

def get_surface_voxels(volume):
    from skimage.measure import marching_cubes
    verts, faces, normals, values = marching_cubes(volume, 0)
    return verts

def compute_surface_distances(surface1, surface2):
    d1 = directed_hausdorff(surface1, surface2)[0]
    d2 = directed_hausdorff(surface2, surface1)[0]
    return d1, d2

def calculate_nsd(ground_truth_file, prediction_file):
    ground_truth = load_nifti(ground_truth_file)
    prediction = load_nifti(prediction_file)

    surface_gt = get_surface_voxels(ground_truth)
    surface_pred = get_surface_voxels(prediction)

    d_gt_pred, d_pred_gt = compute_surface_distances(surface_gt, surface_pred)

    nsd = (d_gt_pred + d_pred_gt) / (len(surface_gt) + len(surface_pred))

    return nsd

# Example usage
ground_truth_file = nii_gt_path
prediction_file = nii_pred_path
nsd_value = calculate_nsd(ground_truth_file, prediction_file)
print(f'Normalized Surface Distance (NSD): {nsd_value}')